# Stock Data Preprocessing Pipeline

This notebook implements a comprehensive preprocessing pipeline for stock data to prepare it for machine learning models including Prophet, LSTM, and XGBoost.

## Pipeline Steps:
1. Fetch daily stock data using yfinance
2. Clean the data (handle missing values, sort by date, etc.)
3. Engineer features (returns, moving averages, technical indicators, etc.)
4. Create target variables for classification
5. Save processed data to CSV files

The processed data will follow the project's structure conventions.

## Import Libraries

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

## Define Stock Tickers

These are the 20 stocks selected for the project.

In [ ]:
tickers = [
    'AAPL',  # Apple Inc.
    'MSFT',  # Microsoft Corporation
    'GOOG',  # Alphabet Inc. (Google)
    'AMZN',  # Amazon.com, Inc.
    'TSLA',  # Tesla, Inc.
    'META',  # Meta Platforms, Inc. (formerly Facebook)
    'NVDA',  # NVIDIA Corporation
    'SPY',   # SPDR S&P 500 ETF Trust
    'V',     # Visa Inc.
    'DIS',   # The Walt Disney Company
    'NFLX',  # Netflix, Inc.
    'PYPL',  # PayPal Holdings, Inc.
    'BABA',  # Alibaba Group
    'IBM',   # International Business Machines Corporation
    'AMD',   # Advanced Micro Devices, Inc.
    'BA',    # The Boeing Company
    'INTC',  # Intel Corporation
    'T',     # AT&T Inc.
    'GS',    # Goldman Sachs Group, Inc.
    'NKE'    # Nike, Inc.
]

## Define Helper Functions

In [ ]:
def fetch_stock_data(ticker, start_date='2010-01-01', end_date=None):
    """
    Fetch stock data for a given ticker using yfinance
    
    Parameters:
    -----------
    ticker : str
        Stock ticker symbol
    start_date : str
        Start date in 'YYYY-MM-DD' format
    end_date : str
        End date in 'YYYY-MM-DD' format, default is today
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame containing stock data
    """
    if end_date is None:
        end_date = datetime.today().strftime('%Y-%m-%d')
        
    stock = yf.Ticker(ticker)
    data = stock.history(start=start_date, end=end_date)
    
    print(f"Fetched {len(data)} rows of data for {ticker} from {start_date} to {end_date}")
    print(data.head(5))
    return data

In [ ]:
def clean_stock_data(df):
    """
    Clean stock data by handling missing values, sorting by date, etc.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing stock data
        
    Returns:
    --------
    pandas.DataFrame
        Cleaned DataFrame
    """
    # Make a copy to avoid modifying the original data
    df = df.copy()
    
    # Reset index to keep Date as a column
    df = df.reset_index()
    
    # Remove timezone info from Date column
    df['Date'] = df['Date'].dt.tz_localize(None)
    
    # Sort data by date (ascending) and reset index
    df = df.sort_values('Date').reset_index(drop=True)
    
    # Drop duplicate rows
    df = df.drop_duplicates(subset=['Date'])
    
    # Replace Volume = 0 with NaN
    df.loc[df['Volume'] == 0, 'Volume'] = np.nan
    
    # Handle missing values: Forward fill for price columns
    for col in ['Open', 'High', 'Low', 'Close']:
        df[col] = df[col].ffill()
    
    # Forward fill for Volume
    df['Volume'] = df['Volume'].ffill()
    
    # Drop any remaining rows with NaNs
    df = df.dropna()
    
    # Ensure integer type for Volume
    df['Volume'] = df['Volume'].astype(int)
    
    # Set Date as index again for feature engineering
    df = df.set_index('Date')
    
    return df

In [ ]:
def add_technical_indicators(df):
    """
    Add technical indicators to the DataFrame
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing stock data
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame with added technical indicators
    """
    df = df.copy()
    
    # Calculate RSI (Relative Strength Index)
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))
    
    # Calculate MACD (Moving Average Convergence Divergence)
    exp1 = df['Close'].ewm(span=12, adjust=False).mean()
    exp2 = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = exp1 - exp2
    df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_Hist'] = df['MACD'] - df['MACD_Signal']
    
    # Calculate Bollinger Bands
    df['MA_20'] = df['Close'].rolling(window=20).mean()
    df['BB_Upper'] = df['MA_20'] + (df['Close'].rolling(window=20).std() * 2)
    df['BB_Lower'] = df['MA_20'] - (df['Close'].rolling(window=20).std() * 2)
    
    # Calculate BB width and position
    df['BB_Width'] = (df['BB_Upper'] - df['BB_Lower']) / df['MA_20']
    df['BB_Position'] = (df['Close'] - df['BB_Lower']) / (df['BB_Upper'] - df['BB_Lower'])
    
    # Calculate Volatility (historical) - Daily returns standard deviation over different windows
    df['Daily_Return'] = df['Close'].pct_change()
    df['Volatility_10D'] = df['Daily_Return'].rolling(window=10).std() * np.sqrt(252)  # Annualized
    df['Volatility_30D'] = df['Daily_Return'].rolling(window=30).std() * np.sqrt(252)  # Annualized
    
    return df

In [ ]:
def engineer_features(df):
    """
    Engineer features for machine learning models
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing clean stock data
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame with engineered features
    """
    df = df.copy()
    
    # --- Technical indicators ---
    df = add_technical_indicators(df)
    
    # --- Returns ---
    # Daily return (already calculated in technical indicators function)
    
    # Weekly return (5 trading days)
    df['Weekly_Return'] = df['Close'].pct_change(5)
    
    # Monthly return (21 trading days)
    df['Monthly_Return'] = df['Close'].pct_change(21)
    
    # --- Moving averages ---
    # MA_20 is already calculated for Bollinger Bands
    df['MA_5'] = df['Close'].rolling(window=5).mean()
    df['MA_10'] = df['Close'].rolling(window=10).mean()
    df['MA_50'] = df['Close'].rolling(window=50).mean()
    df['MA_100'] = df['Close'].rolling(window=100).mean()
    df['MA_200'] = df['Close'].rolling(window=200).mean()
    
    # --- Rolling standard deviation ---
    df['STD_5'] = df['Close'].rolling(window=5).std()
    df['STD_20'] = df['Close'].rolling(window=20).std()
    
    # --- Average volume ---
    df['Volume_MA_20'] = df['Volume'].rolling(window=20).mean()
    
    # --- Price range ---
    df['Price_Range'] = df['High'] - df['Low']
    
    # --- Daily change ---
    df['Daily_Change'] = df['Close'] - df['Open']
    
    # --- Time-based features ---
    # Reset index to get the Date as a column for time-based features
    df = df.reset_index()
    df['DayOfWeek'] = df['Date'].dt.dayofweek
    df['Month'] = df['Date'].dt.month
    
    # Set Date back as index
    df = df.set_index('Date')
    
    return df

In [ ]:
def create_target_variables(df):
    """
    Create target variables for classification
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame with engineered features
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame with target variables
    """
    df = df.copy()
    
    # Target for next day (1 day ahead)
    df['Target_1D'] = np.where(df['Close'].shift(-1) > df['Close'], 1, 0)
    
    # Target for next week (5 trading days ahead)
    df['Target_1W'] = np.where(df['Close'].shift(-5) > df['Close'], 1, 0)
    
    # Target for next month (21 trading days ahead)
    df['Target_1M'] = np.where(df['Close'].shift(-21) > df['Close'], 1, 0)
    
    return df

In [ ]:
def create_directories():
    """
    Create directories for storing processed data
    """
    os.makedirs('../data/cleaned', exist_ok=True)
    print("Created directory: ../data/cleaned")

In [ ]:
def process_stock(ticker, start_date='2010-01-01', end_date=None):
    """
    Process stock data for a given ticker through the entire pipeline
    
    Parameters:
    -----------
    ticker : str
        Stock ticker symbol
    start_date : str
        Start date in 'YYYY-MM-DD' format
    end_date : str
        End date in 'YYYY-MM-DD' format, default is today
        
    Returns:
    --------
    pandas.DataFrame
        Processed DataFrame ready for use in models
    """
    print(f"\n{'='*80}\nProcessing {ticker}\n{'='*80}")
    
    # 1. Fetch data
    df = fetch_stock_data(ticker, start_date, end_date)
    
    # 2. Clean data
    print(f"\nCleaning data for {ticker}...")
    df_clean = clean_stock_data(df)
    print(f"Shape after cleaning: {df_clean.shape}")
    
    # 3. Engineer features
    print(f"\nEngineering features for {ticker}...")
    df_featured = engineer_features(df_clean)
    print(f"Shape after feature engineering: {df_featured.shape}")
    
    # 4. Create target variables
    print(f"\nCreating target variables for {ticker}...")
    df_with_targets = create_target_variables(df_featured)
    print(f"Shape after adding targets: {df_with_targets.shape}")
    
    # 5. Drop rows with NaNs (due to rolling windows and shifting)
    df_final = df_with_targets.dropna()
    print(f"\nFinal shape after dropping NaNs: {df_final.shape}")
    
    # 6. Reset index to make Date a column again before saving
    df_final = df_final.reset_index()
    
    return df_final

## Create Directories

In [ ]:
create_directories()

## Process All Stocks

In [ ]:
def process_all_stocks(tickers, start_date='2010-01-01', end_date=None):
    """
    Process all stocks in the list and save the results to CSV files
    
    Parameters:
    -----------
    tickers : list
        List of stock ticker symbols
    start_date : str
        Start date in 'YYYY-MM-DD' format
    end_date : str
        End date in 'YYYY-MM-DD' format, default is today
    """
    for ticker in tickers:
        try:
            # Process the stock
            df = process_stock(ticker, start_date, end_date)
            
            # Save to CSV
            output_path = f'../data/cleaned/{ticker}.csv'
            df.to_csv(output_path, index=False)
            print(f"\nSaved processed data to {output_path}")
            
        except Exception as e:
            print(f"\nError processing {ticker}: {e}")
            continue

In [ ]:
# Process all stocks (uncomment to run)
process_all_stocks(tickers)

## Process Individual Stock Example

If you want to process just one stock for testing, you can use the code below:

In [ ]:
# Example for processing a single stock (uncomment to run)
ticker = 'AAPL'
df = process_stock(ticker)

# Save to CSV
output_path = f'../data/cleaned/{ticker}.csv'
df.to_csv(output_path, index=False)
print(f"\nSaved processed data to {output_path}")

## Exploratory Analysis of Processed Data

Let's load one of the processed files and examine it to ensure everything looks correct.

In [ ]:
# Uncomment to run after processing at least one stock
ticker = 'AAPL'  # Change this to any processed ticker
file_path = f'../data/cleaned/{ticker}.csv'

if os.path.exists(file_path):
    df = pd.read_csv(file_path)
    
    # Basic info
    print(f"DataFrame shape: {df.shape}")
    print("\nFirst few rows:")
    display(df.head(10))
    
    print("\nColumns:")
    for col in df.columns:
        print(f"- {col}")
    
    # Target distribution
    print("\nTarget distribution:")
    print(f"Target_1D: {df['Target_1D'].value_counts(normalize=True)}")
    print(f"Target_1W: {df['Target_1W'].value_counts(normalize=True)}")
    print(f"Target_1M: {df['Target_1M'].value_counts(normalize=True)}")
else:
    print(f"File {file_path} not found. Please process stocks first.")

## Conclusion

This notebook has implemented a comprehensive preprocessing pipeline for stock data that includes:

1. Fetching historical stock data using yfinance
2. Cleaning the data by handling missing values and timezone information
3. Engineering a rich set of features including:
   - Returns (daily, weekly, monthly)
   - Moving averages
   - Technical indicators (RSI, MACD, Bollinger Bands)
   - Volatility measures
   - Price-based features
   - Time-based features
4. Creating classification target variables for next day, week, and month price direction
5. Saving the processed data to CSV files following the project's naming convention

The processed data is ready for use in machine learning models like Prophet, LSTM, and XGBoost.